In [1]:
# không đủ 17 keypoint
import json
all_path = r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\annotations\person_keypoints_val2017_new.json"
with open(all_path, 'r') as f:
    all_data = json.load(f)
    
for idx, data in enumerate(all_data["annotations"]):
    if len(data["keypoints"])//3 < 17 :
        id = data["image_id"]
        print(f"img id {id}")
    else:
        continue

In [2]:
#convert annotation json to coco 1000 images
import json
import os

for part in range(11,16):
    annotation_path = f"./annotation/part{part}/keypoint.json"                 # annotation
    bbox_path = f"./annotation/part{part}/result.json"                               # bounding box

    output_annotation_path = f"./data/coco/annotations/person_keypoints_val2017_part{part}.json"    # output annotation
    output_bb_path = f"./data/coco/person_detection_results/COCO_val2017_detections_AP_H_56_person_part{part}.json"  # output bounding box

    with open(annotation_path, 'r') as f:
        label_studio_data = json.load(f)

    with open(bbox_path, 'r') as f:
        bb_data = json.load(f)

    coco_output = {
        "info": {
            "description": "Converted from Label Studio",
            "version": "1.0",
            "year": 2024,
            "contributor": "",
            "date_created": ""
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    category_id = 1
    keypoints_mapping = [
            "Nose",
            "Left Eye", 
            "Right Eye", 
            "Left Ear", 
            "Right Ear",
            "Left Shoulder",
            "Right Shoulder",
            "Left Elbow", 
            "Right Elbow", 
            "Left Wrist", 
            "Right Wrist",
            "Left Hip", 
            "Right Hip", 
            "Left Knee", 
            "Right Knee", 
            "Left Ankle", 
            "Right Ankle", 
                
    ]
    coco_output['categories'].append({
        "id": 1,
        "name": "person",
        "supercategory": "person",
        "keypoints": keypoints_mapping,
        "skeleton": []  
    })

    data = []
    sub_id = part*100 - 100 + 1
    annotation_id = sub_id            #fix id
    for idx, item in enumerate(label_studio_data):
        file_name = bb_data["images"][idx]["file_name"]
        image_id = annotation_id
        coco_output['images'].append({
            "id": image_id,
            "file_name": file_name[7:],
            "width": item["annotations"][0]["result"][0]["original_width"],
            "height": item["annotations"][0]["result"][0]["original_height"]
        })

        for annotation in item["annotations"]:
            keypoints = []
            for result in annotation["result"][:17]:
                keypoint = result["value"]
                x = keypoint["x"] / 100.0 * result["original_width"]
                y = keypoint["y"] / 100.0 * result["original_height"]
                keypoints.extend([x, y, 2])
                
            coco_output['annotations'].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "keypoints": keypoints,
                "num_keypoints": len(keypoints) // 3,
                "bbox": bb_data["annotations"][annotation_id-sub_id]["bbox"],       
                "area": bb_data["annotations"][annotation_id-sub_id]["area"]*0.53,  
                "iscrowd": 0
            })
            
            data.append({
                "bbox": bb_data["annotations"][annotation_id-sub_id]["bbox"],      
                "category_id": 1,
                "image_id": image_id,
                "score": 0.99,
            })

            annotation_id += 1

    with open(output_annotation_path, 'w') as f:
        json.dump(coco_output, f, indent=4)

    with open(output_bb_path, 'w') as f:
        json.dump(data, f, indent=4)
        

In [3]:
#merge config annotation and bbox
import json
import os

base_path = 11
output_annotation_path = f"./data/coco/annotations/person_keypoints_val2017_new.json"    # output annotation
output_bb_path = f"./data/coco/person_detection_results/COCO_val2017_detections_AP_H_56_person_new.json"  # output bounding box

annotation_path = f"./data/coco/annotations/person_keypoints_val2017_part{base_path}.json"                 # annotation
bbox_path = f"./data/coco/person_detection_results/COCO_val2017_detections_AP_H_56_person_part{base_path}.json"           # bounding box
with open(annotation_path, 'r') as f:
        label_studio_data = json.load(f)

with open(bbox_path, 'r') as f:
    bb_data = json.load(f)
    
for add_path in range(12,16):

    #add
    annotation_add_path = f"./data/coco/annotations/person_keypoints_val2017_part{add_path}.json"               # annotation
    bbox_add_path = f"./data/coco/person_detection_results/COCO_val2017_detections_AP_H_56_person_part{add_path}.json"           # bounding box
    

    with open(annotation_add_path, 'r') as f:
        label_studio_data_add = json.load(f)

    with open(bbox_add_path, 'r') as f:
        bb_data_add = json.load(f)
        
    label_studio_data["images"].extend(label_studio_data_add["images"])
    label_studio_data["annotations"].extend(label_studio_data_add["annotations"])
    bb_data.extend(bb_data_add)

with open(output_annotation_path, "w") as f:
        json.dump(label_studio_data, f, indent=4)
        
with open(output_bb_path, "w") as f:
        json.dump(bb_data, f, indent=4)

In [4]:
# Create train val test
import random
import json, os
val_dict = {'info': {'description': 'Converted from Label Studio',
  'version': '1.0',
  'year': 2024,
  'contributor': '',
  'date_created': ''},
 'licenses': [],
 'images': [],
 "annotations": None,
 'categories': [{'id': 1,
   'name': 'person',
   'supercategory': 'person',
   'keypoints': ['Nose',
    'Left Eye',
    'Right Eye',
    'Left Ear',
    'Right Ear',
    'Left Shoulder',
    'Right Shoulder',
    'Left Elbow',
    'Right Elbow',
    'Left Wrist',
    'Right Wrist',
    'Left Hip',
    'Right Hip',
    'Left Knee',
    'Right Knee',
    'Left Ankle',
    'Right Ankle'],
   'skeleton': []}]}

test_dict = {'info': {'description': 'Converted from Label Studio',
  'version': '1.0',
  'year': 2024,
  'contributor': '',
  'date_created': ''},
 'licenses': [],
 'images': [],
 "annotations": None,
 'categories': [{'id': 1,
   'name': 'person',
   'supercategory': 'person',
   'keypoints': ['Nose',
    'Left Eye',
    'Right Eye',
    'Left Ear',
    'Right Ear',
    'Left Shoulder',
    'Right Shoulder',
    'Left Elbow',
    'Right Elbow',
    'Left Wrist',
    'Right Wrist',
    'Left Hip',
    'Right Hip',
    'Left Knee',
    'Right Knee',
    'Left Ankle',
    'Right Ankle'],
   'skeleton': []}]}
 

all_path = r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\annotations\person_keypoints_val2017_new.json"
with open(all_path, 'r') as f:
    all_data = json.load(f)

train = all_data.copy()
val_dict['images'] = random.sample(train["images"], 25)
img_id = [val_dict['images'][i]["id"] for i in range(len(val_dict['images']))]
val_dict["annotations"] = [d for d in train["annotations"] if d.get("image_id") in img_id]

images_remove = [image for image in train["images"] if image in val_dict['images']]
anno_remove = [image for image in train["annotations"] if image in val_dict['annotations']]

for image in images_remove:
    train["images"].remove(image)
for image in anno_remove:
    train["annotations"].remove(image)
    
test_dict['images'] = random.sample(train["images"], 25)
img_idx = [test_dict['images'][i]["id"] for i in range(len(test_dict['images']))]
test_dict["annotations"] = [d for d in train["annotations"] if d.get("image_id") in img_idx]


images_remove2 = [image for image in train["images"] if image in test_dict['images']]
anno_remove2 = [image for image in train["annotations"] if image in test_dict['annotations']]

for image in images_remove2:
    train["images"].remove(image)
for image in anno_remove2:
    train["annotations"].remove(image)

with open(r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\annotations\train.json", "w") as f:
        json.dump(train, f, indent=4)
        
with open(r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\annotations\val.json", "w") as f:
        json.dump(val_dict, f, indent=4)
        
with open(r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\annotations\test.json", "w") as f:
        json.dump(test_dict, f, indent=4)



In [5]:
all_path = r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\person_detection_results\COCO_val2017_detections_AP_H_56_person_new.json"
with open(all_path, 'r') as f:
    all_data = json.load(f)
bbox = [data for data in all_data if data.get("image_id") in img_id]
with open(r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\person_detection_results\bbox_val.json", "w") as f:
        json.dump(bbox, f, indent=4)

In [6]:
import json
all_path = r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\person_detection_results\COCO_val2017_detections_AP_H_56_person_new.json"
with open(all_path, 'r') as f:
    all_data = json.load(f)
bbox = [data for data in all_data if data.get("image_id") in img_idx]
with open(r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\person_detection_results\bbox_test.json", "w") as f:
        json.dump(bbox, f, indent=4)

In [7]:
# all_path = r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\person_detection_results\COCO_val2017_detections_AP_H_56_person_new.json"
# with open(all_path, 'r') as f:
#     all_data = json.load(f)
# bbox = [data for data in all_data if data.get("image_id") not in img_id]
# bbox = [data for data in bbox if data.get("image_id") not in img_idx]

# with open(r"D:\Document\End_project_2023\hrnet\Lite-HRNet\data\coco\person_detection_results\bbox_test2.json", "w") as f:
#         json.dump(bbox, f, indent=4)